In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| default_exp helpers

In [ ]:
#| export
import math
import numpy as np

In [ ]:
#| export
import math

class ListChecker:
    @staticmethod
    def check_list_unchanged(float_list, rel_tol=1e-6, abs_tol=0.0):
        if not float_list:  # Check if the list is empty
            return True
        first_value = float_list[0]
        for value in float_list[1:]:
            if not math.isclose(value, first_value, rel_tol=rel_tol, abs_tol=abs_tol):
                return False
        return True

    @staticmethod
    def check_integer_list_unchanged(int_list):
        if not int_list:  # Check if the list is empty
            return True
        first_value = int_list[0]
        for value in int_list[1:]:
            if value != first_value:
                return False
        return True


In [ ]:

# Example usage:
float_list = [1.00000001, 1.00000002, 1.00000003]
int_list = [1, 1, 1]

print(ListChecker.check_list_unchanged(float_list))  # Should print: True (depending on the tolerance)
print(ListChecker.check_integer_list_unchanged(int_list))  # Should print: True


True
True


In [ ]:
#| export
class ARCDataProcessor:
    def __init__(self, config_dict, arc_dict):
        self.arc_dict = arc_dict
        self.grid_shape = self.determine_grid_shape()
        
        if 'grid_shape' in config_dict:
            self.grid_shape = config_dict.get('grid_shape')
        
        self.input_set = config_dict.get('input_set', None)
        if self.input_set not in {'env_only', 'inputs_only', 'both', None}:
            raise ValueError("input_set must be 'env_only', 'inputs_only', 'both', or None")
        
        self.action_set = config_dict.get('action_set', None)
        if self.action_set not in {'dims_only', None}:
            raise ValueError("action_set must be 'dims_only' or None")
        
        self.index = config_dict.get('index', 0)
        self.initial_index = self.index if 'index' in config_dict else None
        self.output_set = config_dict.get('output_set', True)

        if self.action_set == 'dims_only':
            if self.grid_shape is None:
                raise ValueError("grid_shape cannot be None when action_set is 'dims_only'")
            if self.input_set is not None:
                raise ValueError("input_set must be None when action_set is 'dims_only'")

        self.create_env()
        self.info = self.create_info()

    def add_rows(self, num_rows):
        num_rows = round(num_rows)
        self.env = np.pad(self.env, ((0, num_rows), (0, 0)), mode='constant', constant_values=0)

    
    # def remove_rows(self, num_rows):
    #     num_rows = round(num_rows)
    #     if num_rows >= self.env.shape[0]:
    #         num_rows = self.env.shape[0] - 1
    #     if num_rows > 0 and  self.env.shape[0] > num_rows:
    #         self.env = self.env[:-num_rows, :] 

    def remove_rows(self, num_rows):
        num_rows = round(num_rows)
        if num_rows >= self.env.shape[0]:
            num_rows = self.env.shape[0] - 1
        self.env = self.env[:-num_rows, :] if self.env.shape[0] > num_rows else self.env


    def add_columns(self, num_columns):
        num_columns = round(num_columns)
        self.env = np.pad(self.env, ((0, 0), (0, num_columns)), mode='constant', constant_values=0)

    # def remove_columns(self, num_columns):
    #     num_columns = round(num_columns)
    #     if num_columns >= self.env.shape[1]:
    #         num_columns = self.env.shape[1] - 1
    #     if num_columns > 0 and self.env.shape[1] > num_columns :
    #         self.env = self.env[:, :-num_columns] 


    def remove_columns(self, num_columns):
        num_columns = round(num_columns)
        if num_columns >= self.env.shape[1]:
            num_columns = self.env.shape[1] - 1
        self.env = self.env[:, :-num_columns] if self.env.shape[1] > num_columns else self.env


    def process_remaining_values(self, values):
        for i, value in enumerate(values):
            row, col = divmod(i, self.env.shape[1])
            if row < self.env.shape[0] and col < self.env.shape[1]:
                self.env[row, col] = value

    def get_array(self, key, index, sub_key):
        return np.array(self.arc_dict[key][index][sub_key])

    def next(self):
        if self.initial_index is not None:
            return False
        self.index += 1
        self.create_env()
        return True

    def reset(self):
        if self.initial_index is None:
            self.index = 0
        else:
            self.index = self.initial_index
        self.create_env()

    def create_env(self):
        self.env = np.array(self.arc_dict['train'][self.index]['input'])

    def process_dimensions(self, actions):
        if len(actions) == 2:
            num_rows, num_cols = actions
        elif len(actions) == 1:
            num_rows = num_cols = actions[0]
        else:
            raise ValueError("Actions must have one or two values")

        if num_rows > 0:
            self.add_rows(num_rows)
        elif num_rows < 0:
            self.remove_rows(abs(num_rows))

        if num_cols > 0:
            self.add_columns(num_cols)
        elif num_cols < 0:
            self.remove_columns(abs(num_cols))

    def get_input_dimensions(self):
        return np.array(self.arc_dict['train'][self.index]['input']).shape

    def get_output_dimensions(self):
        return np.array(self.arc_dict['train'][self.index]['output']).shape

    def get_env_dimensions(self):
        return self.env.shape

    def apply_actions(self, actions):
        value_index = 0
        if self.grid_shape == 'equal':
            self.process_dimensions(actions[:1])
            value_index = 1
        elif self.grid_shape == 'unequal':
            self.process_dimensions(actions[:2])
            value_index = 2
        
        if self.action_set != 'dims_only':
            self.process_remaining_values(actions[value_index:])

    def create_info(self):
        info = {}
        info['dims'] = 0
        info['num_actions'] = 0
        info['grid_shape'] = self.grid_shape

        if self.grid_shape == 'equal':
            info['num_actions'] = 1
            info['dims'] = 3
        elif self.grid_shape == 'unequal':
            info['num_actions'] = 2
            info['dims'] = 6

        if self.action_set != 'dims_only':
            if self.input_set in {'env_only', 'both'}:
                flattened_env = self.env.flatten()
                info['env'] = len(flattened_env)
                info['num_actions'] += len(flattened_env)
            
            if self.input_set in {'inputs_only', 'both'}:
                flattened_input = self.get_array('train', self.index, 'input').flatten()
                info['inputs'] = len(flattened_input)

            flattened_output = self.get_array('train', self.index, 'output').flatten()
            info['outputs'] = len(flattened_output)

        return info

    def get_state(self):
        values = {}
        self.info['num_actions'] = 0
        if self.grid_shape == 'equal':
            self.info['num_actions'] = 1
            values['env_dims'] = (self.get_env_dimensions()[1],)
            values['input_dims'] = (self.get_input_dimensions()[1],)
            values['output_dims'] = (self.get_output_dimensions()[1],)
        elif self.grid_shape == 'unequal':
            self.info['num_actions'] = 2
            values['env_dims'] = self.get_env_dimensions()
            values['input_dims'] = self.get_input_dimensions()
            values['output_dims'] = self.get_output_dimensions()

        if self.action_set != 'dims_only':
            if self.input_set in {'env_only', 'both'}:
                values['env'] = self.env
                self.info['env'] = self.env.size
                self.info['num_actions'] += self.env.size
            
            if self.input_set in {'inputs_only', 'both'}:
                values['input'] = self.get_array('train', self.index, 'input')
            
            if self.output_set:
                values['output'] = self.get_array('train', self.index, 'output')

        return values, self.info

    def fitness_function(self):
        output_array = np.array(self.arc_dict['train'][self.index]['output'])
        env_array = self.env

        # First metric: square of the difference between the dimensions
        dim_metric = (env_array.shape[0] - output_array.shape[0]) ** 2 + (env_array.shape[1] - output_array.shape[1]) ** 2

        # Second metric: square of the difference between each element in the arrays
        element_metric = 0
        for i in range(max(env_array.shape[0], output_array.shape[0])):
            for j in range(max(env_array.shape[1], output_array.shape[1])):
                env_value = env_array[i, j] if i < env_array.shape[0] and j < env_array.shape[1] else None
                output_value = output_array[i, j] if i < output_array.shape[0] and j < output_array.shape[1] else None
                if env_value is None or output_value is None:
                    element_metric += 25
                else:
                    element_metric += (env_value - output_value) ** 2

        # Final metric
        if self.action_set == 'dims_only':
            final_metric = dim_metric
        else:
            final_metric = dim_metric + element_metric

        return final_metric

    def get_train_input(self):
        return np.array(self.arc_dict['train'][self.index]['input'])

    def get_train_output(self):
        return np.array(self.arc_dict['train'][self.index]['output'])

    def get_test_input(self):
        return np.array(self.arc_dict['test'][0]['input'])  # Assuming only one test case as per the provided format

    def get_info(self):
        return self.info

    def determine_grid_shape(self):
        output_dims = [np.array(task['output']).shape for task in self.arc_dict['train']]
        input_dims = [np.array(task['input']).shape for task in self.arc_dict['train']]

        if len(set(output_dims)) == 1:
            if len(set(input_dims)) == 1 and output_dims[0] == input_dims[0]:
                return None
            return 'equal'
        return 'unequal'

    def get_env_inputs_names(self):
        input_names = []
        if 'dims' in self.info:
            if self.info['dims'] == 2:
                input_names += ['IWE', 'IWO']
            elif self.info['dims'] == 3:
                input_names += ['IWE', 'IWI', 'IWO']
            elif self.info['dims'] == 4:
                input_names += ['IWE', 'IHE', 'IWO', 'IHO']
            elif self.info['dims'] == 6:
                input_names += ['IWE', 'IHE', 'IWI', 'IHI', 'IWO', 'IHO']
            
        if 'env' in self.info:
            num = self.info['env']
            for i in range(num):
                input_names.append(f'IE{i+1:03}')
            
        if 'inputs' in self.info:
            num = self.info['inputs']
            for i in range(num):
                input_names.append(f'II{i+1:03}')

        if 'outputs' in self.info:
            num = self.info['outputs']
            for i in range(num):
                input_names.append(f'IO{i+1:03}')
            
        return input_names

    def get_env_inputs_indexes(self):
        ninputs = 0

        if 'dims' in self.info:
            ninputs += self.info['dims']

        if 'env' in self.info:
            ninputs += self.info['env']

        if 'inputs' in self.info:
            ninputs += self.info['inputs']

        if 'outputs' in self.info:
            ninputs += self.info['outputs']

        env_inputs_indexes = [i for i in range(ninputs)]

        return env_inputs_indexes



In [ ]:

# Example usage:
config_dict = {
    # 'grid_shape': 'unequal',
    'grid_shape': 'equal',
    # 'grid_shape': None,
    # 'input_set': 'env_only',
    # 'input_set': 'both',
    # 'input_set': None,
    'action_set': 'dims_only',
    'index': 0
}

arc_dict = {
    'test': [{'input': [[7, 0, 7], [7, 0, 7], [7, 7, 0]]}],
    'train': [
        {'input': [[0, 7, 7], [7, 7, 7], [0, 7, 7]], 'output': [[0, 0, 0, 0, 7, 7, 0, 7, 7], [0, 0, 0, 7, 7, 7, 7, 7, 7], [0, 0, 0, 0, 7, 7, 0, 7, 7], [0, 7, 7, 0, 7, 7, 0, 7, 7], [7, 7, 7, 7, 7, 7, 7, 7, 7], [0, 7, 7, 0, 7, 7, 0, 7, 7], [0, 0, 0, 0, 7, 7, 0, 7, 7], [0, 0, 0, 7, 7, 7, 7, 7, 7], [0, 0, 0, 0, 7, 7, 0, 7, 7]]}
        # Add more entries as needed
    ]
}

gp = ARCDataProcessor(config_dict, arc_dict)
print('grid_shape', gp.grid_shape)
info = gp.create_info()
print(info)
ins = gp.get_env_inputs_names()
print(ins)

inds = gp.get_env_inputs_indexes()
# print(len(inds))
print(inds)

values, info = gp.get_state()
print(info)
# print(len(values))
print(values)
print('fitness', gp.fitness_function())

actions = [-5]
gp.apply_actions(actions)
values, info = gp.get_state()
print(info)
# print(len(values))
print(values)
print('fitness', gp.fitness_function())

actions = [-5]
gp.apply_actions(actions)
values, info = gp.get_state()
print(info)
# print(len(values))
print(values)
print('fitness', gp.fitness_function())

# actions = [5,4]
# gp.apply_actions(actions)
# values, info = gp.get_state()
# print(info)
# print(len(values))
# print(values)
# print('fitness', gp.fitness_function())


grid_shape equal
{'dims': 3, 'num_actions': 1, 'grid_shape': 'equal'}
['IWE', 'IWI', 'IWO']
[0, 1, 2]
{'dims': 3, 'num_actions': 1, 'grid_shape': 'equal'}
{'env_dims': (3,), 'input_dims': (3,), 'output_dims': (9,)}
fitness 72
{'dims': 3, 'num_actions': 1, 'grid_shape': 'equal'}
{'env_dims': (1,), 'input_dims': (3,), 'output_dims': (9,)}
fitness 128
{'dims': 3, 'num_actions': 1, 'grid_shape': 'equal'}
{'env_dims': (0,), 'input_dims': (3,), 'output_dims': (9,)}
fitness 162


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()